In [ ]:
#import libraries
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv(r'Thesis\data\\once_daily.csv')

Impute missing data with KNN

In [ ]:
from sklearn.impute import KNNImputer



# Create an instance of the KNNImputer with nearest neighbors = 5
imputer = KNNImputer(n_neighbors=5)

# Perform KNN imputation on the numeric columns
data_numeric_imputed = imputer.fit_transform(data)

# Combine the imputed numeric columns with the original date columns

data1 = pd.DataFrame(data_numeric_imputed, columns=data.columns)

Features Engineering

1. Trend slopes of sleep and study hours for all of the months

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

# Group the data by 'StudentID'
grouped_data = data1.groupby('UserID')

# Create an empty dataframe to store the results
result_df = pd.DataFrame(columns=['UserID', 'study_time', 'sleep'])

# Iterate over each group
for student_id, group in grouped_data:
    # Extract the x values for linear regression
    x = np.arange(len(group))  # Assuming time is represented by the index of the dataframe
    
    # Perform linear regression for each variable and calculate the trend slope
    s1 = linregress(x, group['STUDY1']).slope
    s2 = linregress(x, group['hours_slept']).slope


    
    # Append the result to the result dataframe
    result_df = result_df.append({'UserID': student_id,
                                  'study_time': s1,
                                  'sleep': s2,},
                                    ignore_index=True)

In [ ]:
#save the results
result_df.to_csv(r'new_removed_tables\sleep_study_trens_slopes_all_months.csv')

2. Trend slopes of sleep and study hours for months January-March and April-June

In [ ]:
#trend slopes before and after the exams

from scipy.stats import linregress

# Create the first sample from the beginning month to the end of the third month
sample1 = data1[data1['month'].isin([1, 2, 3])]

# Create the second sample from the beginning of the fourth month to the end of the sixth month
sample2 = data1[data1['month'].isin([4, 5, 6])]

# Group the data by 'UserID' in each sample
grouped_data1 = sample1.groupby('UserID')
grouped_data2 = sample2.groupby('UserID')

# Create empty dataframes to store the results for each sample
result_df1 = pd.DataFrame(columns=['StudentID', 'sleep_slope_march', 'study_slope_march'])
result_df2 = pd.DataFrame(columns=['StudentID', 'sleep_slope_june', 'study_slope_june'])

# Iterate over each group in sample 1
for student_id, group in grouped_data1:
    # Extract the x values for linear regression
    x = np.arange(len(group))  # Assuming time is represented by the index of the dataframe
    
    # Perform linear regression for each variable and calculate the trend slope
    s1 = linregress(x, group['hours_slept']).slope
    s2 = linregress(x, group['STUDY1']).slope

    
    # Append the result to the result dataframe for sample 1
    result_df1 = result_df1.append({'StudentID': student_id,'sleep_slope_march': s1, 'study_slope_march': s2 }, ignore_index=True)

# Iterate over each group in sample 2
for student_id, group in grouped_data2:
    # Extract the x values for linear regression
    x = np.arange(len(group))  # Assuming time is represented by the index of the dataframe
    
    # Perform linear regression for each variable and calculate the trend slope
    s3 = linregress(x, group['hours_slept']).slope
    s4 = linregress(x, group['STUDY1']).slope


    result_df2 = result_df2.append({'StudentID': student_id,
                                    'sleep_slope_june': s3,'study_slope_june': s4 }, ignore_index=True)

In [ ]:
#save results
esult_df1.to_csv(r'new_removed_tables\sleep_study_trens_slopes_march.csv')
result_df2.to_csv(r'new_removed_tables\sleep_study_trens_slopes_june.csv')

3. Average hours of sleep per month

In [ ]:

#aggregatee data monthly
grouped = data1.groupby(['UserID', pd.Grouper(key='Date', freq='M')]).apply(np.mean)

In [ ]:
#create a dataframe
df_new = pd.DataFrame(grouped)

In [ ]:
#flatten the table
df_new.reset_index(inplace=True)

In [ ]:
#pivot tables

month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

df_new = pd.pivot_table(df_new, values=df_new.columns, index=df_new['UserID'], columns=['month'])

In [ ]:
df_new = df_new.reindex(month_order, axis=1, level=1)

In [ ]:
#flatten the pivoted table


new_columns = [f"{top_col} {sub_col}" for top_col, sub_col in df_new.columns]

In [ ]:
df_new.columns = new_columns
df_new.reset_index(inplace=True)

In [ ]:
#save table

df_new.to_csv(r'new_removed_tables\sleep_study.csv')